In [1]:
import cv2
import numpy as np
# laneall = cv2.imread("division_end/20180709/153115166085.jpg")
# lane1 = cv2.resize(laneall, (laneall.shape[1],laneall.shape[1]))
laneall = cv2.imread("111222333.jpg")
lane1 = cv2.resize(laneall, (laneall.shape[1],laneall.shape[1]))
# 高斯模糊，Canny边缘检测需要的
lane = cv2.GaussianBlur(lane1, (5, 5), 0)


In [2]:
def unevenLightCompensate(img, blockSize):# 光照均衡化，过度地带颜色偏深
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    average = np.mean(gray)

    rows_new = int(np.ceil(gray.shape[0] / blockSize))
    cols_new = int(np.ceil(gray.shape[1] / blockSize))

    blockImage = np.zeros((rows_new, cols_new), dtype=np.float32)
    for r in range(rows_new):
        for c in range(cols_new):
            rowmin = r * blockSize
            rowmax = (r + 1) * blockSize
            if (rowmax > gray.shape[0]):
                rowmax = gray.shape[0]
            colmin = c * blockSize
            colmax = (c + 1) * blockSize
            if (colmax > gray.shape[1]):
                colmax = gray.shape[1]

            imageROI = gray[rowmin:rowmax, colmin:colmax]
            temaver = np.mean(imageROI)
            blockImage[r, c] = temaver

    blockImage = blockImage - average
    blockImage2 = cv2.resize(blockImage, (gray.shape[1], gray.shape[0]), interpolation=cv2.INTER_CUBIC)
    gray2 = gray.astype(np.float32)
    dst = gray2 - blockImage2
    dst = dst.astype(np.uint8)
    dst = cv2.GaussianBlur(dst, (3, 3), 0)

    return dst


In [3]:
# blockSize = 16
# lane1 = unevenLightCompensate(lane, blockSize)
# cv2.imshow("123", lane1)
# cv2.waitKey()

In [4]:
# lane1 = cv2.flip(lane1, 1, dst=None)
# cv2.imshow("123", lane1)
# cv2.waitKey()

In [5]:

# kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))  # 定义结构元素
# kerne2 = cv2.getStructuringElement(cv2.MORPH_RECT, (7, 7))
# opening = cv2.morphologyEx(lane, cv2.MORPH_OPEN, kernel)  # 开运算
# lane = cv2.morphologyEx(opening, cv2.MORPH_CLOSE, kerne2)

# 进行边缘检测，减少图像空间中需要检测的点数量
lane = cv2.Canny(lane1, 30, 100)
print(lane.shape)
cv2.imshow("lane", lane)
cv2.waitKey(0)

(201, 201)


-1

In [6]:

# lines = cv2.HoughLines(edges, 4, np.pi/180,200)
# lines = np.reshape(lines,(lines.shape[0],-1))
rho = 1  # 距离分辨率
theta = np.pi / 180  # 角度分辨率
threshold = 10  # 霍夫空间中多少个曲线相交才算作正式交点
min_line_len = 10  # 最少多少个像素点才构成一条直线
max_line_gap = 50  # 线段之间的最大间隔像素
lines = cv2.HoughLinesP(lane, rho, theta, threshold, maxLineGap=max_line_gap)
line_img = np.zeros_like(lane)

point_x = []
point_y = []
point_distence = []
for line in lines:
    for x1, y1, x2, y2 in line:
        point_x.append(x1)
        point_x.append(x2)
        point_y.append(y1)
        point_y.append(y2)
        point_distence.append(x1*x1+(y1+10)**2)
        point_distence.append(x2*x2+(y2+10)**2)
#         cv2.line(line_img, (x1, y1), (x2, y2), 255, 1)
max_index = point_distence.index(max(point_distence))
min_index = point_distence.index(min(point_distence))
cv2.line(line_img, (point_x[max_index], point_y[max_index]), (point_x[min_index], point_y[min_index]), 255, 1)
cv2.imshow("line_img", line_img)
cv2.waitKey()

x1y1 = [point_x[min_index], point_y[min_index]]
x4y4 = [point_x[max_index], point_y[max_index]]

for i in range(len(point_distence)):
    point_y[i] = lane.shape[0] - point_y[i]
    point_distence[i] = point_x[i]*point_x[i]+point_y[i]*point_y[i]
max_index = point_distence.index(max(point_distence))
min_index = point_distence.index(min(point_distence))
cv2.line(line_img, (point_x[max_index], lane.shape[0] - point_y[max_index]), (point_x[min_index], lane.shape[0] - point_y[min_index]), 255, 1)
cv2.imshow("line_img", line_img)
cv2.waitKey()

x3y3 = [point_x[min_index], lane.shape[0] - point_y[min_index]]
x2y2 = [point_x[max_index], lane.shape[0] - point_y[max_index]]


print(x1y1, x2y2, x3y3, x4y4)


[26, 0] [199, 5] [20, 183] [196, 198]


In [7]:
pts1 = np.float32([x1y1,x2y2,x3y3,x4y4])
pts2 = np.float32([[0, 0], [lane.shape[1], 0], [0, lane.shape[0]*0.7], [lane.shape[1], lane.shape[0]*0.7]])
M = cv2.getPerspectiveTransform(pts1, pts2)
dst = cv2.warpPerspective(lane1, M, (lane.shape[1], lane.shape[0]))
endd = dst[0:int(lane.shape[0]*0.65), :]
cv2.imshow("end", endd)
cv2.waitKey()

-1

In [8]:
cv2.imwrite("end.jpg", endd)

True

In [9]:
# pts1 = np.float32([x1y1,x2y2,x3y3,x4y4])
# pts2 = np.float32([[0, 0], [lane.shape[1], 0], [0, lane.shape[0]], [lane.shape[1], lane.shape[0]]])
# M = cv2.getPerspectiveTransform(pts1, pts2)
# dst = cv2.warpPerspective(lane1, M, (lane.shape[1], lane.shape[0]))
# dst = cv2.resize(dst,(laneall.shape[1], laneall.shape[0]))
# cv2.imshow("end", dst)
# cv2.waitKey()